<a href="https://colab.research.google.com/github/kimdonggyu2008/SKT_FLY_AI_6/blob/main/%ED%81%AC%EB%A1%A4%EB%A7%81_%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import ssl

# ✅ SSL 인증서 검증 비활성화 설정
ssl_context = ssl._create_unverified_context()
# ✅ 공공데이터포털 API 서비스 URL (입찰 정보)
api_url = "http://apis.data.go.kr/1230000/ad/BidPublicInfoService/getBidPblancListInfoCnstwk"

# ✅ 서비스 키 설정 (URL 디코딩)
service_key = unquote("Mpk%2Bcqgr7klV97OB0fZVbHnLy5nawJk3fg4Fhy5qYvHnSQC8SlGHoAElBwFZ3DsoA%2FHlIz9pzZAlc1f%2Bo48mtg%3D%3D")  # 서비스 키를 올바르게 디코딩

# ✅ 요청 파라미터 설정
queryParams = "?" + urlencode({
    quote_plus("serviceKey"): service_key,
    quote_plus("pageNo"): "1",  # 페이지 번호
    quote_plus("numOfRows"): "10000",  # 한 페이지당 결과 개수
    quote_plus("inqryDiv"): "1",  # 조회 구분
    quote_plus("inqryBgnDt"): "201601010000",  # 조회 시작일
    quote_plus("type"): "json",  # JSON 응답 요청
    quote_plus("inqryEndDt"): "201601300000",  # 조회 종료일

}, quote_via=quote_plus)

# ✅ 최종 요청 URL
request_url = api_url + queryParams

# ✅ API 요청
request = Request(request_url)
request.get_method = lambda: 'GET'  # GET 요청 방식 지정
response_body = urlopen(request).read().decode("utf-8")  # 응답 데이터 읽기 및 UTF-8 디코딩

# ✅ 결과 출력
print(response_body)


{"response": {
  "header": {
    "resultCode": "00",
    "resultMsg": "정상"
  },
  "body": {
    "items": [
      {
        "bidNtceNo": "20151229288",
        "bidNtceOrd": "001",
        "reNtceYn": "N",
        "rgstTyNm": "조달청 또는 나라장터 자체 공고건",
        "ntceKindNm": "변경공고",
        "intrbidYn": "N",
        "bidNtceDt": "2016-01-01 16:37:40",
        "refNo": "부천시공고 제2015-1710호",
        "bidNtceNm": "상상거리 지역경제활성화 야간경관조성[긴급]",
        "ntceInsttCd": "3860000",
        "ntceInsttNm": "경기도 부천시",
        "dminsttCd": "3860000",
        "dminsttNm": "경기도 부천시",
        "bidMethdNm": "전자입찰",
        "cntrctCnclsMthdNm": "일반경쟁",
        "ntceInsttOfclNm": "박서윤",
        "ntceInsttOfclTelNo": "032-625-2651",
        "ntceInsttOfclEmailAdrs": "vezanua83@korea.kr",
        "exctvNm": "박서윤",
        "bidQlfctRgstDt": "2016-01-07 18:00",
        "cmmnSpldmdAgrmntRcptdocMethd": "공고서참조",
        "cmmnSpldmdAgrmntClseDt": "",
        "cmmnSpldmdCorpRgnLmtYn": "N",
        "bidBeginDt": "2016-01-01 

# 전체내용 크롤링


In [ ]:
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import ssl
import pandas as pd
from datetime import datetime, timedelta
import time
import os
import csv
import json

# ✅ SSL 인증서 검증 비활성화 설정
ssl_context = ssl._create_unverified_context()

# ✅ 공공데이터포털 API 서비스 URL (입찰 정보)
api_url = "http://apis.data.go.kr/1230000/ad/BidPublicInfoService/getBidPblancListInfoCnstwk"

# ✅ 서비스 키 설정 (URL 디코딩)
service_key = unquote("Mpk%2Bcqgr7klV97OB0fZVbHnLy5nawJk3fg4Fhy5qYvHnSQC8SlGHoAElBwFZ3DsoA%2FHlIz9pzZAlc1f%2Bo48mtg%3D%3D")

# ✅ CSV 저장 파일명
csv_filename = "bid_info_mural_filtered.csv"

# ✅ 시작 날짜 (2016-01-01)부터 현재 날짜까지 15일씩 증가
start_date = datetime(2016, 1, 1)
end_date = datetime.today()

first_run = True  # CSV 파일 헤더를 한 번만 저장하기 위한 플래그

while start_date < end_date:
    # ✅ 15일 후 날짜 설정
    next_date = start_date + timedelta(days=30)

    # ✅ 날짜 포맷 변환 (yyyyMMddHHmm)
    inqryBgnDt = start_date.strftime("%Y%m%d%H%M")
    inqryEndDt = next_date.strftime("%Y%m%d%H%M")

    page_no = 1  # ✅ 페이지 번호 초기화

    while True:  # ✅ 모든 페이지를 가져올 때까지 반복
        # ✅ 요청 파라미터 설정
        queryParams = "?" + urlencode({
            quote_plus("serviceKey"): service_key,
            quote_plus("pageNo"): str(page_no),  # ✅ 페이지 번호
            quote_plus("numOfRows"): "100",  # 한 페이지당 결과 개수
            quote_plus("inqryDiv"): "1",  # 조회 구분
            quote_plus("inqryBgnDt"): inqryBgnDt,  # 조회 시작일
            quote_plus("inqryEndDt"): inqryEndDt,  # 조회 종료일
            quote_plus("type"): "json",  # JSON 응답 요청
        }, quote_via=quote_plus)

        # ✅ 최종 요청 URL
        request_url = api_url + queryParams
        print(f"🔍 요청 중: {inqryBgnDt} ~ {inqryEndDt} (페이지 {page_no})")

        # ✅ API 요청
        try:
            request = Request(request_url)
            request.get_method = lambda: 'GET'  # GET 요청 방식 지정
            response_body = urlopen(request, context=ssl_context).read().decode("utf-8")  # SSL 검증 비활성화 후 요청

            # ✅ JSON 데이터 파싱
            json_return = json.loads(response_body)

            # ✅ API 응답에서 데이터 추출
            items = json_return.get("response", {}).get("body", {}).get("items", [])
            print(len(items))

            if not items:
                print(f"⚠️ {inqryBgnDt} ~ {inqryEndDt} (페이지 {page_no}) 데이터 없음, 종료")
                break  # ✅ 더 이상 데이터가 없으면 종료

            # ✅ "벽화" 키워드가 포함된 데이터 필터링
            filtered_items = [item for item in items if ("벽화" in item.get("bidNtceNm", "") or True)]

            if filtered_items:
                df = pd.DataFrame(filtered_items)

                # ✅ CSV 파일에 저장 (헤더는 첫 번째 실행에서만 추가)
                if first_run:
                    df.to_csv(csv_filename, index=False, mode="w", encoding="utf-8-sig")
                    first_run = False  # 헤더 저장 이후 변경
                else:
                    df.to_csv(csv_filename, index=False, mode="a", encoding="utf-8-sig", header=False)

                print(f"✅ {inqryBgnDt} ~ {inqryEndDt} (페이지 {page_no}) 벽화 데이터 추가 완료 ({len(filtered_items)}건)")

            # ✅ 다음 페이지로 이동
            page_no += 1

            # ✅ 요청 간격을 늘려 API 요청 제한 방지 (1초 대기)
            time.sleep(1)

        except Exception as e:
            print(f"\n❌ API 요청 오류 ({inqryBgnDt} ~ {inqryEndDt}, 페이지 {page_no}): {e}")
            break  # ✅ 에러 발생 시 페이지 반복 중단

    # ✅ 다음 날짜로 이동
    start_date = next_date

print(f"\n✅ 벽화 관련 데이터 저장 완료: {csv_filename}")


🔍 요청 중: 201601010000 ~ 201601310000 (페이지 1)
10
✅ 201601010000 ~ 201601310000 (페이지 1) 벽화 데이터 추가 완료 (10건)
🔍 요청 중: 201601010000 ~ 201601310000 (페이지 2)
10
✅ 201601010000 ~ 201601310000 (페이지 2) 벽화 데이터 추가 완료 (10건)
🔍 요청 중: 201601010000 ~ 201601310000 (페이지 3)
10
✅ 201601010000 ~ 201601310000 (페이지 3) 벽화 데이터 추가 완료 (10건)
🔍 요청 중: 201601010000 ~ 201601310000 (페이지 4)
10
✅ 201601010000 ~ 201601310000 (페이지 4) 벽화 데이터 추가 완료 (10건)


KeyboardInterrupt: 

# 벽화관련 프로젝트만 크롤링

In [3]:
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import ssl
import pandas as pd
import time
import json
from datetime import datetime, timedelta
import os

# ✅ SSL 인증서 검증 비활성화 설정
ssl_context = ssl._create_unverified_context()

# ✅ 공공데이터포털 API 서비스 URL (입찰 정보)
api_url = "http://apis.data.go.kr/1230000/ad/BidPublicInfoService/getBidPblancListInfoCnstwkDtl"

# ✅ 서비스 키 설정 (URL 디코딩)
service_key = unquote("xAI5Cagagsyktl%2B5DkdMACQ3R0c%2FseI5XjAcTth2misYLb5QZbxp0OZqYnCJ3%2FtGvlIeG6Ny0AjEszLbWitP3w%3D%3D")

# ✅ 저장할 CSV 파일명
output_csv_filename = "/mnt/data/bid_info_filtered.csv"

# ✅ CSV 파일에서 bidNtceNo(입찰 공고 번호) 불러오기
input_csv_filename = "/content/drive/MyDrive/Bid_Notice_Numbers.csv"
df_bid = pd.read_csv(input_csv_filename)

# ✅ 공고 번호 리스트 추출
bid_numbers = df_bid["bidNtceNo"].tolist()
print(f"📌 총 {len(bid_numbers)}개의 공고 번호를 조회합니다.")

# ✅ CSV 파일이 없을 경우 헤더를 추가하여 생성
first_run = not os.path.exists(output_csv_filename)

# ✅ 공고 번호별로 상세 입찰 정보 가져오기
detailed_data = []
for bidNtceNo in bid_numbers:
    bidNtceOrd = "1"  # ✅ 기본값 1로 설정

    # ✅ 공고 번호에서 연도와 월 추출하여 조회 기간 자동 설정 (형식 검증 포함)
    try:
        # ✅ 공고 번호의 길이 확인 (정상적인 공고 번호는 8자리 이상)
        if len(bidNtceNo) < 8 or not bidNtceNo[:6].isdigit():
            print(f"⚠️ 잘못된 공고 번호 형식: {bidNtceNo} → 스킵")
            continue  # 잘못된 공고 번호는 건너뛴다.

        year = int(bidNtceNo[:4])  # ✅ 공고 번호 앞 4자리에서 연도 추출
        month = int(bidNtceNo[4:6])  # ✅ 5~6자리에서 월 추출

        # ✅ 유효한 연도와 월인지 검증 (예: 2000~2025년 사이, 월은 1~12)
        if year < 2000 or year > 2025 or month < 1 or month > 12:
            print(f"⚠️ 잘못된 연도 또는 월: {bidNtceNo} → 스킵")
            continue  # 잘못된 날짜 형식이면 무시

        # ✅ 조회 시작일: 해당 연도, 월의 첫날
        inqryBgnDt = datetime(year, month, 1).strftime("%Y%m%d%H%M")

        # ✅ 조회 종료일: 해당 연도, 월의 마지막 날
        last_day = (datetime(year, month, 1) + timedelta(days=31)).replace(day=1) - timedelta(days=1)
        inqryEndDt = last_day.strftime("%Y%m%d%H%M")

    except Exception as e:
        print(f"❌ 공고 번호 {bidNtceNo}에서 연도, 월 추출 실패: {e} → 스킵")
        continue  # 잘못된 데이터는 스킵

    # ✅ 요청 파라미터 설정 (사용자가 지정한 순서 유지)
    queryParams = "?" + urlencode([
        (quote_plus("serviceKey"), service_key),
        (quote_plus("pageNo"), "1"),  # ✅ 페이지 번호
        (quote_plus("numOfRows"), "10000"),  # ✅ 한 페이지당 결과 개수
        (quote_plus("inqryDiv"), "1"),  # ✅ 조회 구분
        (quote_plus("inqryBgnDt"), inqryBgnDt),  # ✅ 자동 설정된 조회 시작일
        (quote_plus("type"), "json"),  # ✅ JSON 응답 요청
        (quote_plus("inqryEndDt"), inqryEndDt),  # ✅ 자동 설정된 조회 종료일
        (quote_plus("bidNtceNo"), bidNtceNo)  # ✅ 공고 번호
    ], quote_via=quote_plus)

    # ✅ 최종 요청 URL
    request_url = api_url + queryParams
    print(request_url)
    print(f"🔍 요청 중: 공고 번호 {bidNtceNo}, 조회 기간: {inqryBgnDt} ~ {inqryEndDt}")

    try:
        # ✅ API 요청
        request = Request(request_url)
        request.get_method = lambda: 'GET'
        response_body = urlopen(request, context=ssl_context).read().decode("utf-8")

        # ✅ JSON 데이터 파싱
        json_return = json.loads(response_body)

        # ✅ API 응답에서 데이터 추출
        detail_item = json_return.get("response", {}).get("body", {}).get("items", {})

        if detail_item:
            detail_item["bidNtceNo"] = bidNtceNo  # ✅ 공고 번호 추가
            detailed_data.append(detail_item)
            print(f"✅ 공고 번호 {bidNtceNo} 세부 데이터 저장 완료.")

        else:
            print(f"⚠️ 공고 번호 {bidNtceNo}에 대한 데이터 없음.")

        # ✅ API 요청 간격 조정 (0.5초 대기)
        time.sleep(0.5)

    except Exception as e:
        print(f"❌ 세부 정보 요청 오류 (공고 번호: {bidNtceNo}): {e}")

# ✅ 데이터 저장
if detailed_data:
    df_detailed = pd.DataFrame(detailed_data)
    df_detailed.to_csv(output_csv_filename, index=False, encoding="utf-8-sig")
    print(f"\n✅ 모든 공고의 세부 정보 저장 완료: {output_csv_filename}")


📌 총 605개의 공고 번호를 조회합니다.
http://apis.data.go.kr/1230000/ad/BidPublicInfoService/getBidPblancListInfoCnstwkDtl?serviceKey=xAI5Cagagsyktl%2B5DkdMACQ3R0c%2FseI5XjAcTth2misYLb5QZbxp0OZqYnCJ3%2FtGvlIeG6Ny0AjEszLbWitP3w%3D%3D&pageNo=1&numOfRows=10000&inqryDiv=1&inqryBgnDt=201612010000&type=json&inqryEndDt=201612310000&bidNtceNo=20161224957-000
🔍 요청 중: 공고 번호 20161224957-000, 조회 기간: 201612010000 ~ 201612310000
❌ 세부 정보 요청 오류 (공고 번호: 20161224957-000): HTTP Error 500: Internal Server Error
http://apis.data.go.kr/1230000/ad/BidPublicInfoService/getBidPblancListInfoCnstwkDtl?serviceKey=xAI5Cagagsyktl%2B5DkdMACQ3R0c%2FseI5XjAcTth2misYLb5QZbxp0OZqYnCJ3%2FtGvlIeG6Ny0AjEszLbWitP3w%3D%3D&pageNo=1&numOfRows=10000&inqryDiv=1&inqryBgnDt=201611010000&type=json&inqryEndDt=201611300000&bidNtceNo=20161124515-000
🔍 요청 중: 공고 번호 20161124515-000, 조회 기간: 201611010000 ~ 201611300000
❌ 세부 정보 요청 오류 (공고 번호: 20161124515-000): HTTP Error 500: Internal Server Error
http://apis.data.go.kr/1230000/ad/BidPublicInfoService/get

KeyboardInterrupt: 

In [34]:
import requests
import pandas as pd
import time
import json
from datetime import datetime, timedelta

# ✅ API 기본 설정
api_url = "http://apis.data.go.kr/1230000/ad/BidPublicInfoService/getBidPblancListInfoServc"
service_key = "xAI5Cagagsyktl+5DkdMACQ3R0c/seI5XjAcTth2misYLb5QZbxp0OZqYnCJ3/tGvlIeG6Ny0AjEszLbWitP3w=="  # 🔴 실제 서비스 키 입력 필요

# ✅ 입찰공고번호 CSV 불러오기
input_csv_filename = "/content/drive/MyDrive/Bid_Notice_Numbers.csv"
df_bid = pd.read_csv(input_csv_filename)

# ✅ 공고 번호 리스트 추출
bid_numbers = df_bid["bidNtceNo"].astype(str).tolist()
print(f"📌 총 {len(bid_numbers)}개의 공고 번호를 조회합니다.")

# ✅ 결과 저장 리스트
detailed_data = []

# ✅ 입찰공고번호별 API 요청
for bidNtceNo in bid_numbers:
    try:
        # ✅ 공고번호에서 연, 월, 일 추출 (형식: YYYYMMDD0000)
        if len(bidNtceNo) < 8:
            print(f"⚠️ 잘못된 공고 번호: {bidNtceNo} → 스킵")
            continue

        year = int(bidNtceNo[:4])
        month = int(bidNtceNo[4:6])
        day = int(bidNtceNo[6:8])

        # ✅ 조회 시작일: 해당 공고번호 날짜 00:00:00
        inqryBgnDt = datetime(year, month, day).strftime("%Y%m%d0000")

        # ✅ 조회 종료일: 해당 공고번호 날짜 23:59:59
        inqryEndDt = datetime(year, month, day).strftime("%Y%m%d2359")

        # ✅ API 요청 파라미터 설정
        params = {
            "serviceKey": service_key,
            "type": "json",
            "pageNo": "3",
            "numOfRows": "999",
            "inqryDiv": "1",  # 🔍 개찰결과 조회
            "inqryBgnDt": "201612200000",
            "inqryEndDt": "201612202359",

            #"inqryBgnDt": inqryBgnDt,
            #"inqryEndDt": inqryEndDt,
            "bidNtceNo": bidNtceNo
        }

        # ✅ 최종 요청 URL 확인 (출력)
        request_url = requests.Request('GET', api_url, params=params).prepare().url
        print(f"🔗 요청 URL: {request_url}")  # 📌 요청 URL 출력

        # ✅ API 요청
        response = requests.get(request_url)
        response.raise_for_status()
        json_return = response.json()

        # ✅ API 응답에서 데이터 추출
        items = json_return.get("response", {}).get("body", {}).get("items", None)

        # ✅ 데이터 처리
        if isinstance(items, list):
            for item in items:
                item["bidNtceNo"] = bidNtceNo
                detailed_data.append(item)
        elif isinstance(items, dict):
            items["bidNtceNo"] = bidNtceNo
            detailed_data.append(items)
        else:
            print(f"⚠️ 데이터 없음: {bidNtceNo}")

        # ✅ API 요청 간격 조정 (과부하 방지)
        time.sleep(0.5)

    except Exception as e:
        print(f"❌ 오류 발생 (공고번호 {bidNtceNo}): {e}")

# ✅ 데이터 저장
if detailed_data:
    df_detailed = pd.DataFrame(detailed_data)

    # ✅ 필요한 컬럼만 선택하여 저장
    selected_columns = [
        "bidNtceNo", "bidNtceNm", "ntceInsttNm", "bidMethdNm",
        "cntrctCnclsMthdNm", "bidBeginDt", "bidClseDt", "opengDt",
        "presmptPrce", "bdgtAmt", "cnstrtsiteRgnNm"
    ]
    df_detailed = df_detailed[[col for col in selected_columns if col in df_detailed.columns]]

    # ✅ CSV 파일로 저장
    output_csv_filename = "/content/final_bid_info_filtered.csv"
    df_detailed.to_csv(output_csv_filename, index=False, encoding="utf-8-sig")
    print(f"\n✅ 저장 완료: {output_csv_filename}")

else:
    print("⚠️ 저장할 데이터가 없습니다.")


📌 총 605개의 공고 번호를 조회합니다.
🔗 요청 URL: http://apis.data.go.kr/1230000/ad/BidPublicInfoService/getBidPblancListInfoServc?serviceKey=xAI5Cagagsyktl%2B5DkdMACQ3R0c%2FseI5XjAcTth2misYLb5QZbxp0OZqYnCJ3%2FtGvlIeG6Ny0AjEszLbWitP3w%3D%3D&type=json&pageNo=3&numOfRows=999&inqryDiv=1&inqryBgnDt=201612200000&inqryEndDt=201612202359&bidNtceNo=20161224957-000
🔗 요청 URL: http://apis.data.go.kr/1230000/ad/BidPublicInfoService/getBidPblancListInfoServc?serviceKey=xAI5Cagagsyktl%2B5DkdMACQ3R0c%2FseI5XjAcTth2misYLb5QZbxp0OZqYnCJ3%2FtGvlIeG6Ny0AjEszLbWitP3w%3D%3D&type=json&pageNo=3&numOfRows=999&inqryDiv=1&inqryBgnDt=201612200000&inqryEndDt=201612202359&bidNtceNo=20161124515-000
🔗 요청 URL: http://apis.data.go.kr/1230000/ad/BidPublicInfoService/getBidPblancListInfoServc?serviceKey=xAI5Cagagsyktl%2B5DkdMACQ3R0c%2FseI5XjAcTth2misYLb5QZbxp0OZqYnCJ3%2FtGvlIeG6Ny0AjEszLbWitP3w%3D%3D&type=json&pageNo=3&numOfRows=999&inqryDiv=1&inqryBgnDt=201612200000&inqryEndDt=201612202359&bidNtceNo=20161121152-000
🔗 요청 URL: http://ap

KeyboardInterrupt: 